## 1. Import libraries

In [1]:
import numpy as np 
import pandas as pd 
import sklearn 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import (
    OneHotEncoder
)
from feature_engine.encoding import RareLabelEncoder

## 2. Display Setting

In [2]:
pd.set_option("display.max_columns",None)

In [3]:
#sklearn will return dataframe
sklearn.set_config(transform_output= "pandas") 

In [4]:
# warnings.filterwarnings("ignore")

## 3.Read the data

In [5]:
file_path = r"/Users/sudhirjoon/Library/Mobile Documents/com~apple~CloudDocs/Uni_Mannheim/Nitish/Project_1_Flight_price_prediction/Data/train.csv"
train = pd.read_csv(file_path)
train

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-12,Delhi,Cochin,20:55:00,19:00:00,1325,1.0,In-flight meal not included,10262
1,Jet Airways,2019-05-27,Banglore,Delhi,15:15:00,18:10:00,175,0.0,In-flight meal not included,3502
2,Jet Airways,2019-04-01,Kolkata,Banglore,20:25:00,18:00:00,1295,1.0,No Info,11467
3,Jet Airways,2019-04-01,Kolkata,Banglore,09:35:00,21:05:00,690,1.0,No Info,10953
4,Spicejet,2019-04-01,Banglore,Delhi,05:55:00,08:35:00,160,0.0,No check-in baggage included,3919
...,...,...,...,...,...,...,...,...,...,...
635,Air India,2019-04-24,Banglore,Delhi,13:20:00,16:10:00,170,0.0,No Info,6121
636,Indigo,2019-04-01,Mumbai,Hyderabad,09:10:00,10:40:00,90,0.0,No Info,3342
637,Indigo,2019-06-27,Chennai,Kolkata,14:45:00,17:05:00,140,0.0,No Info,3597
638,Indigo,2019-03-06,Mumbai,Hyderabad,02:30:00,04:00:00,90,0.0,No Info,3175


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   airline          640 non-null    object 
 1   date_of_journey  640 non-null    object 
 2   source           640 non-null    object 
 3   destination      640 non-null    object 
 4   dep_time         640 non-null    object 
 5   arrival_time     640 non-null    object 
 6   duration         640 non-null    int64  
 7   total_stops      640 non-null    float64
 8   additional_info  640 non-null    object 
 9   price            640 non-null    int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 50.1+ KB


In [7]:
X_train = train.drop(columns= "price")
y_train = train.price.copy()

## 4. Transformation Operations

### 4.1 airline

In [8]:
X_train.airline

0      Jet Airways
1      Jet Airways
2      Jet Airways
3      Jet Airways
4         Spicejet
          ...     
635      Air India
636         Indigo
637         Indigo
638         Indigo
639    Jet Airways
Name: airline, Length: 640, dtype: object

In [9]:
X_train.columns.to_list()

['airline',
 'date_of_journey',
 'source',
 'destination',
 'dep_time',
 'arrival_time',
 'duration',
 'total_stops',
 'additional_info']

In [10]:
air_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="most_frequent")),
	("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
	("encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

air_transformer.fit_transform(X_train.loc[:, ["airline"]])#.airline.value_counts()

,airline_Air India,airline_Indigo,airline_Jet Airways,airline_Multiple Carriers,airline_Other
0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...
635,1.0,0.0,0.0,0.0,0.0
636,0.0,1.0,0.0,0.0,0.0
637,0.0,1.0,0.0,0.0,0.0
638,0.0,1.0,0.0,0.0,0.0
